<a href="https://colab.research.google.com/github/c-c-c-c/dm_integration/blob/master/myMecab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 吉村インテグレーションステップ 

目的は、ドラマの構成/脚本として、「どのタイミングで」「どんな
出来事」が起こると良いかという示唆を見出すこと。

★目的変数
　　ドラマの初回視聴率からの上下動(%で、閾値を良い、悪い、
　　普通になるように３パターン準備)

★特徴量
　　-ドラマを恋愛、刑事、ヒューマンなどのカテゴリーに分ける
　　-さらに、ドラマをステージ(序盤、中盤、終盤)に分ける
　　-ドラマ名-ステージを行にした、単語×ドラマステージ行列を作成

★モデル
　　- 目的変数に対しロッソ回帰を行う。
　　- もしくは、教師なしのグルーピングを行う。(kmeans, トピックモデル)

In [125]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [126]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.10-6ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.5)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.8)
xz-utils is already installed at the requested version (5.2.2-1.3)
file is already installed at the requested version (1:5.32-2ubuntu0.3)
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+mai

In [127]:
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

fatal: destination path 'mecab-ipadic-neologd' already exists and is not an empty directory.
[install-mecab-ipadic-NEologd] : Start..
[install-mecab-ipadic-NEologd] : Check the existance of libraries
[install-mecab-ipadic-NEologd] :     find => ok
[install-mecab-ipadic-NEologd] :     sort => ok
[install-mecab-ipadic-NEologd] :     head => ok
[install-mecab-ipadic-NEologd] :     cut => ok
[install-mecab-ipadic-NEologd] :     egrep => ok
[install-mecab-ipadic-NEologd] :     mecab => ok
[install-mecab-ipadic-NEologd] :     mecab-config => ok
[install-mecab-ipadic-NEologd] :     make => ok
[install-mecab-ipadic-NEologd] :     curl => ok
[install-mecab-ipadic-NEologd] :     sed => ok
[install-mecab-ipadic-NEologd] :     cat => ok
[install-mecab-ipadic-NEologd] :     diff => ok
[install-mecab-ipadic-NEologd] :     tar => ok
[install-mecab-ipadic-NEologd] :     unxz => ok
[install-mecab-ipadic-NEologd] :     xargs => ok
[install-mecab-ipadic-NEologd] :     grep => ok
[install-mecab-ipadic-NEo

In [0]:
import joblib
import MeCab
import numpy as np
import pandas as pd
import re
import json
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso


from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.svm import LinearSVC

## データの読み込み

In [0]:
# EPGデータ(手作業での修正中))
df = pd.read_excel("./drive/My Drive/0_インテグ作業/data/EPG_checking0212.xlsx")

In [0]:
f= open("./drive/My Drive/0_インテグ作業/data/drama_category0220.json", 'r')

drama_category_dic = json.load(f) #JSON形式で読み込む



In [0]:
f= open("./drive/My Drive/0_インテグ作業/data/drama_win_lose.json", 'r')

drama_win_lose_dic = json.load(f) #JSON形式で読み込む



In [132]:
df.columns

Index(['Unnamed: 0', 'drama_key', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'drama_title', 'sharp_title', 'sharp_cnt', 'sharp_cnt_epg_regex',
       'start_time', 'sharp_epg', 'sharp_epg_hand_corrected', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
       'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24'],
      dtype='object')

In [0]:
df["sharp_epg_tknz"] = np.nan

In [0]:
# ゴミ除去

def removeTrash (text):
    import re

    result_text = text
    result_text = re.sub(r"https?://[\w/:%#\$&\?\(\)~\.=\+\-]+", "", result_text)
    result_text = re.sub(r"番組詳細|制作・著作|制作著作", "", result_text)
    result_text = re.sub(r"[!\(\)=『』～/]", "", result_text)
    result_text = re.sub(r"フジテレビ|日本テレビ|TBS|テレビ朝日|TBS|関西テレビ", "", result_text)
    result_text = re.sub(r"\d+", "", result_text)
    
    result_text = re.sub(r"【公式.*?】", "", result_text)
    result_text = re.sub(r"\u3000", "", result_text)

    return result_text


In [135]:
mecab = MeCab.Tagger()
mecab.parse("")
for i, text in enumerate( df['sharp_epg_hand_corrected']):
    text_tokenized = []
    # print(i)

    # URL、記号などのゴミを取り除く
    if type(text) is not str: 
        if np.isnan(text) :
            continue 
    text = removeTrash(text)
    node = mecab.parseToNode(text)
    while node:
        node = node.next
        if node is None:
            continue

        if not node.feature.startswith("BOS/EOS") and not node.feature.startswith("助詞") and\
            not node.feature.startswith("記号") and\
            node.feature.find("人名") == -1 and\
            not node.feature.startswith("助動詞"):
            text_tokenized.append(node.surface)

    df["sharp_epg_tknz"].iloc[i] = text_tokenized

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
# 手作業での前処理が完了してないので、完了したものだけにする
bool_list = [] 

for i in range(len(df["sharp_epg_hand_corrected"])):
    bool_list.append( type( df["sharp_epg_tknz"].iloc[i] ) != float )

In [0]:
df_notnull =  df[bool_list]

In [138]:
df_notnull["phys_cnt"] = np.nan
df_notnull["category"] = np.nan
df_notnull =  df_notnull.rename(columns={ 'Unnamed: 0' : 'sort_id' } )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [139]:
drama_category_dic.keys()

dict_keys(['love', 'police'])

In [141]:
# 物理カウントとカテゴリーを加える
love_cnt = 0
police_cnt = 0
for tmp_key  in df_notnull["drama_key"].unique():
    cnt = 0

    if tmp_key in drama_category_dic["love"]:
        print("love☆")
        love_cnt += 1

        qry_l = " drama_key == @tmp_key"
        target_idx = df_notnull.query(qry_l).index
        df_notnull[ "category" ].loc [target_idx] = "love"


    if tmp_key in drama_category_dic["police"]:
        police_cnt += 1
        qry_p = " drama_key == @tmp_key"
        target_idx = df_notnull.query(qry_p).index
        df_notnull[ "category" ].loc [target_idx] = "police"
        # print(police_cnt)

    for sort_id in  df_notnull[df_notnull["drama_key"] == tmp_key]['sort_id'].values:
        cnt  += 1

        qry = " sort_id == @sort_id"
        target_idx = df_notnull.query(qry).index
        df_notnull["phys_cnt"].loc[target_idx] = cnt

    print(tmp_key)



/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


1910_CX_月21
1910_CX_火22
love☆
1910_CX_木22
1907_CX_木22
1904_CX_月21
love☆
1904_CX_火21
1904_CX_木22
1901_CX_月21
1901_CX_火21
1901_CX_木22
1810_CX_月21
1810_CX_火21
1810_CX_木22
1807_CX_月21
1807_CX_火21
1807_CX_木22
1804_CX_月21
1804_CX_火21
1804_CX_木22
love☆
1801_CX_月21
1801_CX_木22
1710_CX_月21
1710_CX_火21
1710_CX_木22
1707_CX_火21
1707_CX_木22
1707_CX_日21
1704_CX_月21
1704_CX_火21
1704_CX_木22
1704_CX_日21
love☆
1701_CX_月21
1701_CX_火21
1701_CX_木22
1701_CX_日21
love☆
1610_CX_月21
1610_CX_火21
1610_CX_木22
1610_CX_日21
love☆
1607_CX_月21
1607_CX_火22
1607_CX_日21
love☆
1604_CX_月21
1604_CX_火22
1604_CX_木22
1604_CX_土23
love☆
1601_CX_月21
1601_CX_火22
1601_CX_水22
1601_CX_木22
love☆
1510_CX_月21
1510_CX_火22
1510_CX_水22
love☆
1510_CX_木22
love☆
1507_CX_月21
1507_CX_火22
1507_CX_水22
1507_CX_木22
1504_CX_月21
love☆
1504_CX_水22
1504_CX_木22
love☆
1501_CX_月21
1501_CX_火21
1501_CX_火22
1501_CX_水22
1501_CX_木22
1410_CX_月21
1410_CX_火21
1410_CX_火22
1410_CX_水22
1410_CX_木22
1407_CX_月21
1407_CX_火21
1407_CX_火22
1407_CX_水22
love☆
1407_CX_木22
1407

In [142]:
df_notnull[df_notnull["category"] == "love"].drama_key.unique()

array(['1910_CX_木22', '1904_CX_火21', '1801_CX_月21', '1701_CX_月21',
       '1610_CX_月21', '1607_CX_月21', '1604_CX_月21', '1601_CX_月21',
       '1510_CX_月21', '1510_CX_木22', '1507_CX_月21', '1504_CX_水22',
       '1501_CX_月21', '1407_CX_木22', '1404_CX_木22', '1401_CX_月21',
       '1210_CX_木22', '1207_CX_土23', '1201_CX_木22', '1110_CX_月21',
       '1110_CX_木22', '1107_CX_月21', '1104_CX_月21', '1101_CX_月21',
       '1010_CX_月21', '1007_CX_月21', '1004_CX_月21', '1004_CX_木22',
       '0907_CX_月21', '0907_CX_火22', '0907_CX_土23', '0804_CX_火21',
       '0804_CX_火22', '0801_CX_月21', '0707_CX_月21', '1910_EX_土23',
       '1904_EX_土23', '1901_EX_土23', '1810_EX_土23', '1804_EX_土23',
       '1707_EX_金23', '1701_EX_金23', '1604_EX_金23', '1501_EX_金23',
       '1004_EX_木21', '0907_EX_木21', '1907_NTV_水22', '1907_NTV_土22',
       '1810_NTV_水22', '1807_NTV_水22', '1807_NTV_土22', '1804_NTV_木23',
       '1801_NTV_日22', '1707_NTV_水22', '1707_NTV_日22', '1704_NTV_木23',
       '1704_NTV_土22', '1701_NTV_水22', '1604_NTV_水22

In [0]:
# tmp_key = "1807_TBS_日21"
# qry = "drama_key == @tmp_key & phys_cnt < 4 "
# str(df_notnull.query(qry)["sharp_epg_hand_corrected"].values)

In [0]:
# df_stage ステージごとに作る

df_stage = pd.DataFrame( columns=['drama_key','drama_title','stages','epg_joined','drama_category','win_lose'] )


for tmp_key in df_notnull[df_notnull["category"] == "love"].drama_key.unique():

    win_or_lose = ""
    if tmp_key in drama_win_lose_dic["win"]:
        win_or_lose = "win"
    elif tmp_key in drama_win_lose_dic["draw"]:
        win_or_lose = "draw"
    elif tmp_key in drama_win_lose_dic["lose"]:
        win_or_lose = "lose"
    else:
        print("????")

        # target_idx = df_stage.query(qry_wl).index
        # df_notnull[ "category" ].loc [target_idx] = "police"
        # # print(police_cnt)


    for tmp_stage in ["early","middle" ,"late"]:

        qry = ""
        if tmp_stage == "early":
            qry = "drama_key == @tmp_key & phys_cnt < 4 "

        elif tmp_stage == "middle":
            qry = "drama_key == @tmp_key & 4< phys_cnt < 7 "

        else:
            qry = "drama_key == @tmp_key & phys_cnt >= 7 "

        target_i =  df_notnull.query(qry)["sharp_epg_hand_corrected"]
        epg_joined = str(df_notnull.query(qry)["sharp_epg_hand_corrected"].values)

        tmp_se = pd.Series( [ tmp_key, "..title" ,tmp_stage, epg_joined  ,"love", win_or_lose ]  , index=df_stage.columns  )
        df_stage = df_stage.append( tmp_se, ignore_index=True )

# early, middle, lateで作る

# for i in range():
    




In [145]:
df_stage


,drama_key,drama_title,stages,epg_joined,drama_category,win_lose
0,1910_CX_木22,..title,early,['現在失業中の27歳独身・難波ユリカ(新木優子)は、見た目にも気を使い、コミュニケーション...,love,lose
1,1910_CX_木22,..title,middle,['マコチ(高良健吾)は、ユリカ(新木優子)のもとへ謝りに行ったものの、追い返されてしまう。...,love,lose
2,1910_CX_木22,..title,late,['ユリカ(新木優子)は、マコチ(高良健吾)から「好きだ」と告白され、もう一度彼と付き合うこ...,love,lose
3,1904_CX_火21,..title,early,['12年ぶりの再会は偶然か、それとも運命かーー。恋をあきらめた鮎川樹(松坂桃李)と、初恋相...,love,win
4,1904_CX_火21,..title,middle,['つぐみ(山本美月)が駅のホームから転落した。車いすから懸命に手を伸ばすが届かず、線路に落...,love,win
5,1904_CX_火21,..title,late,['松本に戻って3カ月、つぐみ(山本美月)はインテリアコーディネーターとして働きながら、実家...,love,win
6,1801_CX_月21,..title,early,"['クラゲを愛しすぎてしまった筋金入りの“クラゲオタク女子""が、とある兄弟と三角関係になり、...",love,lose
7,1801_CX_月21,..title,middle,['稲荷翔子(泉里香)が『天水館』の持ち主である千絵子(富山えり子)の母に直接、土地建物の売...,love,lose
8,1801_CX_月21,..title,late,"['修(工藤阿須加)から告白された月海(芳根京子)は大混乱。“尼～ず""に相談出来ない月海は、...",love,lose
9,1701_CX_月21,..title,early,['高梨あすか(西内まりや)はいずみ銀行の法人営業課で働く銀行員。金融に関する数々の資格を持...,love,lose


In [0]:
# 空の列を足す
df_stage["epg_tknz"] = np.nan

In [147]:
mecab = MeCab.Tagger()
mecab.parse("")
for i, text in enumerate( df_stage['epg_joined']):
    text_tokenized = []
    # print(i)

    # URL、記号などのゴミを取り除く
    if type(text) is not str: 
        if np.isnan(text) :
            continue 
    text = removeTrash(text)
    node = mecab.parseToNode(text)
    while node:
        node = node.next
        if node is None:
            continue

        if not node.feature.startswith("BOS/EOS") and not node.feature.startswith("助詞") and\
            not node.feature.startswith("記号") and\
            node.feature.find("人名") == -1 and\
            not node.feature.startswith("助動詞"):
            text_tokenized.append(node.surface)

    df_stage["epg_tknz"].iloc[i] = text_tokenized

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [148]:
df_stage["epg_tknz"]

0      [[', 現在, 失業, 中, 歳, 独身, 難波, ユリカ, 見た目, 気, 使い, コミ...
1      [[', マコチ, ユリカ, もと, 謝り, 行っ, 追い返さ, れ, しまう, 失意, マ...
2      [[', ユリカ, マコチ, 好き, 告白, さ, れ, もう一度, 彼, 付き合う, こと...
3      [[', 年, ぶり, 再会, 偶然, それとも, 運命, ーー, 恋, あきらめ, 松坂,...
4      [[', つぐみ, 駅, ホーム, 転落, し, 車いす, 懸命, 手, 伸ばす, 届か, ...
5      [[', 松本, 戻っ, カ月, つぐみ, インテリア, コーディネーター, 働き, 実家,...
6      [[', クラゲ, 愛し, すぎ, しまっ, 筋金入り, クラゲ, オタク, 女子, ", ...
7      [[', 稲荷, 天水, 館, 持ち主, 母, 直接, 土地, 建物, 売買, 契約, 持ち...
8      [[', 工藤, 阿須, 加, 告白, さ, れ, 月, 海, 芳, 根, 大, 混乱, 尼...
9      [[', 高, 梨, あすか, いずみ, 銀行, 法人, 営業, 課, 働く, 銀行, 員,...
10     [[', 打ち合わせ, 終え, あすか, 三郎, 甘, 路, マンション, 送っ, もらう,...
11     [[', 名波, 同棲, 不倫, 二股, 疑惑, 伝える, ゴシップ, 報じ, られ, あす...
12     [['「, 総合, 地所, 株式会社, 創立, 周年, 記念, パーティー, 行わ, れ, ...
13     [[', 殴ら, れる, 手掛ける, バンコク, プロジェクト, 助, 出資, 優, 頼ん,...
14     [[', 禁断, 愛, 揺れる, 梓, 結婚, 控え, 優, 想い, てん, びん, 均衡,...
15     [[', フジ, 月, 好き, 人, いる, こと, 公式, パティシエ, 一流, ホテル,...
16     [[', フジ, 月, 好き, 人, いる, こと, 公式, 平, 花火, 大会, 行く, ...
17     [[', フジ, 月, 好き, 人, いる, こ

In [149]:
df_stage["win_lose_dummy"] = np.nan
for i in range(len( df_stage["win_lose"] )):

    if df_stage["win_lose"].iloc[i] == "win":
        df_stage["win_lose_dummy"].iloc[i] = 1

    elif df_stage["win_lose"].iloc[i] == "draw":
        df_stage["win_lose_dummy"].iloc[i] = 0

    elif df_stage["win_lose"].iloc[i] == "lose":
        df_stage["win_lose_dummy"].iloc[i] = -1
        
    else :
        print("???")
    # df_stage["win_lose_dummy"] = 

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
# 行列の中で1より大きい数値を１にならす
def bool_bow ( argX ) :

    for i in range(argX.shape[0]):
        # print(type(Xe[i]))
        for j in range(argX.shape[1]):
            if argX[i, j] > 1:
                argX[i, j] = 1

    return argX

In [0]:
# ステージごとに分けた後のdfを引数にとる

def fit_trans_lasso (df) :
    bow_vectorizer = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
    X = bow_vectorizer.fit_transform(
        [str(i) for i in df["epg_tknz"].values]
    )
    X = bool_bow(X)

    Y = df["win_lose_dummy"]

    scaler = StandardScaler(with_mean=False)
    clf = Lasso(alpha=0.1)

    scaler.fit(X)
    clf.fit(scaler.transform(X), Y)

    result_df = pd.DataFrame(clf.coef_.T , index=bow_vectorizer.vocabulary_ )
    return result_df


In [0]:
class HorizontalDisplay:
    def __init__(self, *args):
        self.args = args

    def _repr_html_(self):
        template = '<div style="float: left; padding: 10px;">{0}</div>'
        return "\n".join(template.format(arg._repr_html_())
                         for arg in self.args)


In [0]:
#######
# early middle lateステージごとの結果を出す
#

def calc_lasso_by_stage (df):
    df_early =  df[df["stages"] == "early"]
    df_middle =  df[df["stages"] == "middle"]
    df_late =  df[df["stages"] == "late"]

    Xe = fit_trans_lasso(df_early)
    Xm = fit_trans_lasso(df_middle)
    Xl = fit_trans_lasso(df_late)

    result_early = fit_trans_lasso (df_early) 
    result_middle = fit_trans_lasso (df_middle) 
    result_late = fit_trans_lasso (df_late) 

    import IPython.core.display as display
    import IPython.display


    import pprint
    # pprint.pprint(result_early[ result_early[0] != 0].sort_values(0).T)
    # pprint.pprint(result_middle[ result_middle[0] != 0].sort_v).T)alues(0).T)
    # pprint.pprint(result_late[ result_late[0] != 0].sort_values(0



    dfe = result_early[ result_early[0] != 0].sort_values(0)
    dfm =result_middle[ result_middle[0] != 0].sort_values(0)
    dfl =result_late[ result_late[0] != 0].sort_values(0)



    # print
    display.display(HorizontalDisplay(dfe, dfm, dfl))




In [0]:
# df_early = df_stage[df_stage["stages"] == "early"]

In [0]:
# df_middle = df_stage[df_stage["stages"] == "middle"]

In [156]:
calc_lasso_by_stage(df_stage)



,0
削減,-1.197294e-01
ナガクラ,-1.055826e-01
学期,-7.145511e-02
思慮,-6.104342e-02
在中,-5.877837e-02
ずぶ濡れ,-5.532519e-02
快く,-5.252574e-02
議題,-5.205934e-02
悩む,-4.706868e-02
生き方,-4.427682e-02


In [0]:
# df_late = df_stage[df_stage["stages"] == "late"]

In [0]:
# pd.DataFrame(Xe.toarray(), columns=[ x[0] for x in sorted(bow_vectorizer_early.vocabulary_.items(), key=lambda x: x[1]) ])

In [0]:
#####################
#
#以下、授業の残り
#
####################



### 簡単な例

In [0]:
# ※これは、演習用に単語文書行列を DataFrame に変換して見やすくしてみるためのコードで、覚える必要はありません
pd.DataFrame(X.toarray(), columns=[ x[0] for x in sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1]) ])